In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from src.models.load_data import Balance, Instruments, AdvisedPortfolios, PriceDB, Singleton

In [5]:
access_path = './data/processed/'
# bal_from_nm = 'bal_btw_rebal_20210602.dat'
last_rebal_nm = 'balance_rebaldate_0702_all.dat'

In [6]:
bal_from = pd.read_csv(access_path + last_rebal_nm)
price_db = PriceDB.instance().data

In [8]:
price_db

,date,itemcode,price,volume,trading_amt_mln,ret,itemtype
0,2018-07-20 00:00:00,A069500,27910.0000,8517238.0,237716.112580,NaN,ETF
1,2018-07-21 00:00:00,A069500,27910.0000,8517238.0,237716.112580,NaN,ETF
2,2018-07-22 00:00:00,A069500,27910.0000,8517238.0,237716.112580,NaN,ETF
3,2018-07-23 00:00:00,A069500,27685.0000,7283838.0,219684.583805,-0.008094,ETF
4,2018-07-24 00:00:00,A069500,27770.0000,2562621.0,170177.717593,0.003066,ETF
...,...,...,...,...,...,...,...
38565,2021-07-31,A283580,13670.0000,45156.0,782.000976,-0.005835,ETF
38566,2021-07-31,A292150,14605.0000,314427.0,5492.303664,-0.009879,ETF
38567,2021-07-31,A293180,42455.0000,16183.0,2589.006318,-0.012872,ETF
38568,2021-07-31,A302190,52745.0000,243.0,113.671938,0.000759,ETF


#### MP 발생일
- 2021.5.3
- 2021.6.2
- 2021.7.2
- 2021.8.1 (일요일)

In [87]:
# 두 MP발생일 사이 기간 설정
start_dt = '20210703'  # MP발생 다음날부터
end_dt = '20210731'    # MP발생 전날까지

range_dt_ymd = pd.date_range(start=start_dt, end=end_dt, freq='D')  # price_db 날짜 포맷
range_dt_mdy = range_dt_ymd.strftime('%-m/%-d/%Y')  # 잔고(balance) 날짜 포맷

In [88]:
# price_db 마지막 날짜를 휴일에 복사하기
# temp = price_db[price_db.date==price_db.iloc[-1,0]].copy()
# temp.date = '2021-07-31'
# price_db = pd.concat((price_db, temp), axis=0)
# price_db = price_db.reset_index(drop=True)

#### 최근 과거 MP발생일 잔고를 해당 기간 내 매일 매일의 잔고로 복사해 두고, price 컬럼과 value 컬럼을 업데이트하면 끝.

- 잔고 복사

In [54]:
df_list = []
for cnt, dt in enumerate(range_dt_ymd):
    temp = bal_from.copy(deep=True)
    temp.date = dt
    df_list.append(temp)
    
bal_btw_rebal = pd.concat(df_list)

- `price_db`를 이용해서 Price 컬럼 업데이트.

In [55]:
bal_btw_rebal = bal_btw_rebal.set_index(['date', 'itemcode'])
price_db = price_db.set_index(['date', 'itemcode'])
bal_btw_rebal.update(price_db)
bal_btw_rebal = bal_btw_rebal.reset_index(drop=False)

- 사용자별/날짜별로 비중(`wt`) 계산

In [56]:
bal_btw_rebal.value = bal_btw_rebal.quantity*bal_btw_rebal.price
bal_btw_rebal.original = ''
bal_btw_rebal.wt = bal_btw_rebal.value/bal_btw_rebal.groupby(['date', 'userid'])['value'].transform('sum')

In [57]:
bal_btw_rebal

,date,itemcode,userid,name,asset_class,itemname,quantity,cost_price,cost_value,price,value,wt,group_by,original
0,2021-07-03,A114260,A001,안정추구형소규모,Fixed Income,KODEX 국고채3년,2.0,57845.0,115690.0,57790.0,115580.0,0.162089,NaN,
1,2021-07-03,A122260,A001,안정추구형소규모,Fixed Income,KOSEF 통안채1년,1.0,101120.0,101120.0,101115.0,101115.0,0.141804,NaN,
2,2021-07-03,A130730,A001,안정추구형소규모,Fixed Income,KOSEF 단기자금,2.0,101060.0,202120.0,101060.0,202120.0,0.283453,NaN,
3,2021-07-03,A196230,A001,안정추구형소규모,Fixed Income,KBSTAR 단기통안채,1.0,104430.0,104430.0,104435.0,104435.0,0.146460,NaN,
4,2021-07-03,A273130,A001,안정추구형소규모,Fixed Income,KODEX 종합채권(AA-이상)액티브,1.0,108635.0,108635.0,108665.0,108665.0,0.152392,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083,2021-07-31,A157490,A009,적극투자형대규모,Equity,TIGER 소프트웨어,17.0,19565.0,332605.0,19150.0,325550.0,0.241752,NaN,
2084,2021-07-31,A266370,A009,적극투자형대규모,Equity,KODEX IT,9.0,21175.0,190575.0,20860.0,187740.0,0.139415,NaN,
2085,2021-07-31,A069500,A009,적극투자형대규모,Equity,KODEX 200,3.0,43640.0,130920.0,42485.0,127455.0,0.094647,NaN,
2086,2021-07-31,A114260,A009,적극투자형대규모,Fixed Income,KODEX 국고채3년,1.0,57845.0,57845.0,58030.0,58030.0,0.043093,NaN,


In [58]:
bal_btw_rebal.to_csv(access_path + 'bal_btw_rebal.dat', index=False, encoding='utf-8-sig')

In [59]:
bal_btw_rebal.columns

Index(['date', 'itemcode', 'userid', 'name', 'asset_class', 'itemname',
       'quantity', 'cost_price', 'cost_value', 'price', 'value', 'wt',
       'group_by', 'original'],
      dtype='object')